In [1]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics import accuracy_score


In [2]:
import numpy as np
import pandas as pd

A)Read the dataset file “Cancer.csv” (from github using the following command), and assign it to a Pandas DataFrame:

df  = pd.read_csv("https://github.com/mpourhoma/CS4661/raw/master/Cancer.csv") Check out the dataset. As you see, the dataset includes 9 numerical features. The last column is the binary label (“1” means it is a malignant cancer, “0” means it is a benign tumor). You will use all 9 features in this homework.

In [3]:
cancer_df = pd.read_csv("https://github.com/mpourhoma/CS4661/raw/master/Cancer.csv")

In [4]:
cancer_df.head()

,Clump_Thickness,Uniformity_of_Cell_Size,Uniformity_of_Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Malignant_Cancer
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [5]:
feature_cols = ['Clump_Thickness','Uniformity_of_Cell_Size','Uniformity_of_Cell_Shape',
                'Marginal_Adhesion','Single_Epithelial_Cell_Size','Bare_Nuclei',
                'Bland_Chromatin','Normal_Nucleoli','Mitoses']
X = cancer_df[feature_cols] 

In [6]:
y = cancer_df['Malignant_Cancer']

B)Use sklearn functions to split the dataset into testing and training sets with the following parameters: test_size=0.35, random_state=3.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=3)


print(X_train.shape)
print(y_train.shape)


print(X_test.shape)

print(y_test.shape)

(97, 9)
(97,)
(53, 9)
(53,)


C)Use “Decision Tree Classifier” to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier.

Use this command to define your tree: my_DecisionTree = DecisionTreeClassifier(random_state=3).

In [8]:
from sklearn.tree import DecisionTreeClassifier
my_DecisionTree = DecisionTreeClassifier(random_state=3)
my_DecisionTree.fit(X_train, y_train)
y_predict_dt = my_DecisionTree.predict(X_test)
accu_dt = accuracy_score(y_test, y_predict_dt)

print('Accuracy of decisiontree:',accu_dt)

Accuracy of decisiontree: 0.8301886792452831


 D)Now, we want to perform a new Ensemble Learning method called “Bagging” based on Voting on 19 decision tree classifiers.

Note: you should write your own code to perform Bagging (don’t use scikit-learn functions for Bagging!) To do so, you need to perform bootstrapping first. You can write a “for” loop with loop variable  i = 0…18.  

In each iteration of the loop, you have to:

1. make a bootstarp sample of the original “Training” Dataset (build in part(b)) with the size of bootstarp_size = 0.8*(Size of the original training dataset). You can use the following command to generate a random bootstrap dataset (“i" is the variable of the loop, so the random_state changes in each iteration):

resample(X_train, n_samples = bootstarp_size , random_state=i , replace = True)

2. Define and train a new base decision tree classifier on this dataset in each iteration:

Base_DecisionTree = DecisionTreeClassifier(random_state=3).

3. Perform prediction using “this base classifier” on the original “Testing” Dataset X_test (build in part(b)), and save the prediction results for all testing samples.

After finishing the “for” loop, you should have 19 different predictions for EACH sample in your testing set. Then, Perform Voting to make the final decision on each data sample based on the votes of all 19 classifiers.

Finally, calculate and report the final accuracy of your Bagging (Voting) method.

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from statistics import mode
from scipy import stats
from itertools import chain
prediction =[]
y_predict_voting=[]
bootstrap_size=int(0.8*(X_train.shape[0]))
print("samples")

for i in range(19):
    new_X_train, new_Y_train=resample(X_train, y_train,n_samples = bootstrap_size , random_state=i, replace=True)
    Base_DecisionTree= DecisionTreeClassifier(random_state=3)
    Base_DecisionTree.fit(new_X_train, new_Y_train)
    y_predict_bdt=Base_DecisionTree.predict(X_test)
    prediction.append(y_predict_bdt)
    print("sample",i,":",'',prediction[i])
    print("\n")
predictionMatrix=np.transpose(np.matrix(prediction))
mode_list=stats.mode(predictionMatrix,axis=1)
y_predict_voting = mode_list[0].tolist()

print("mode of each",list(chain(*y_predict_voting)))
accuracy_list = accuracy_score(y_test,y_predict_voting)
print("final accuracy",accuracy_list)

samples
sample 0 :  [1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 0 0
 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 1]


sample 1 :  [1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0
 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1]


sample 2 :  [1 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 1 1 1 1 0 1 1]


sample 3 :  [0 1 0 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0
 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 1]


sample 4 :  [1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 1]


sample 5 :  [1 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0
 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1]


sample 6 :  [1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0
 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 1]


sample 7 :  [0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0
 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1]


sample 8

E)Use scikit-learn “Random Forest” classifier to predict Cancer based on the training/testing datasets that you built in part (b). Then, calculate and report the accuracy of your classifier. Use this command to import and define your classifier:

from   sklearn.ensemble    import    RandomForestClassifier

my_RandomForest = RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=3)

Similar to previous syntax, use my_RandomForest.fit  for training your random forest classifier and my_RandomForest.predict  for prediction.

In [10]:
from   sklearn.ensemble    import    RandomForestClassifier

my_RandomForest = RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=3)
my_RandomForest.fit(X_train,y_train)
y_predict_rf = my_RandomForest.predict(X_test)
accu_rf = accuracy_score(y_test, y_predict_rf)
print('Accuracy of RandomForest:',accu_rf)

Accuracy of RandomForest: 0.9245283018867925
